In [1]:
import os
import re
import docx
import pandas as pd
import openpyxl

# Функция для извлечения текста из документа .docx
def extract_text_from_docx(file_path):
    try:
        print(f"Пытаюсь открыть файл: {file_path}")
        doc = docx.Document(file_path)
        print("Файл успешно открыт.")
        full_text = []
        for paragraph in doc.paragraphs:
            full_text.append(paragraph.text)
        print("Текст успешно извлечён.")
        return '\n'.join(full_text)
    except Exception as e:
        print(f"Ошибка при открытии или чтении файла: {e}")
        return None

# Функция для разделения текста на главы по ключевому слову "Chapter"
def split_text_into_chapters(text):
    if not text:
        print("Текст пуст, невозможно разделить на главы.")
        return {}
    
    print("Разделяю текст на главы...")
    chapters = {}
    chapter_pattern = re.compile(r'Chapter\s+(\d+)', re.IGNORECASE)
    chapter_matches = list(chapter_pattern.finditer(text))
    
    if not chapter_matches:
        print("Не удалось найти главы.")
        return {}

    for i, match in enumerate(chapter_matches):
        start = match.start()
        chapter_title = match.group(0)
        chapter_number = int(match.group(1))  # Извлекаем числовое значение главы
        end = chapter_matches[i+1].start() if i+1 < len(chapter_matches) else len(text)
        chapter_text = text[start:end].strip()
        chapters[chapter_number] = (chapter_title, chapter_text)
    
    print(f"Текст успешно разделён на {len(chapters)} глав.")
    return chapters

# Функция для поиска слов в каждой главе
def search_words_in_chapter(chapter_text, words):
    word_pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    
    matches = []
    for match in re.finditer(word_pattern, chapter_text, re.IGNORECASE):
        start = match.start()
        end = match.end()
        
        # Берем по три слова до и после
        start_context = chapter_text[:start].split()[-8:]
        end_context = chapter_text[end:].split()[:8]
        
        # Получаем найденное слово
        found_word = chapter_text[start:end]
        
        # Формируем результат
        result = ' '.join(start_context + [found_word] + end_context)
        matches.append(result)
    
    return matches

# Функция для поиска слов во всех главах
def search_in_all_chapters(chapters, words, category):
    if not chapters:
        print("Нет глав для поиска.")
        return []
    
    print(f"Ищу заданные слова для категории {category}...")
    results = []
    for chapter_number, (chapter_title, chapter_text) in chapters.items():
        matches = search_words_in_chapter(chapter_text, words)
        if matches:
            for match in matches:
                results.append([chapter_number, chapter_title, match, category])
    print(f"Найдено совпадений в {len(results)} главах для категории {category}.")
    return results

# Основная функция
def main(docx_file_path, chapters_folder):
    # Проверяем, существует ли файл .docx
    if not os.path.exists(docx_file_path):
        print(f"Файл не найден: {docx_file_path}")
        return

    # Извлекаем текст из документа docx
    print(f"Извлекаю текст из {docx_file_path}...")
    book_text = extract_text_from_docx(docx_file_path)
    
    if not book_text:
        print("Ошибка: текст не был извлечён.")
        return

    # Разделяем текст на главы
    chapters = split_text_into_chapters(book_text)
    
    if not chapters:
        print("Ошибка: главы не были найдены.")
        return

    # Разделяем слова по категориям
    clothes_words = ['stilettos', 'cigar', 'cigarette', 'smoking', 'baseball cap', 'hat', 'mask', 'sunglasses', 'blouse', 'tuxedo', 'ring', 'necklace', 'bracelet', 'earrings', 'brooch', 'watch', 'anklet', 'choker', 'pendant', 'cufflinks', 'tie clip', 'nose ring', 'belly button ring', 'toe ring', 'hairpin', 'tiara', 'diadem', 'bangle', 'chain', 'medallion', 'pearl necklace', 'locket', 'armband', 'charm bracelet', 'dress', 'robe', 'suit', 'clothes', 'coat', 'jacket', 'shirt', 'pants', 'skirt', 'jeans', 't-shirt', 'sweater', 'blouse', 'shorts', 'hoodie', 'vest', 'scarf', 'hat', 'gloves', 'boots', 'shoes', 'sneakers', 'socks', 'tie', 'belt', 'gown', 'trench coat', 'blazer', 'cardigan', 'overalls', 'tank top', 'leggings']
    hair_words = ['hair', 'beard', 'brunette', 'ironed', 'straightened', 'locks', 'flowing', 'ponytail', 'bun', 'braids', 'bob', 'pixie cut', 'long waves', 'curly hair', 'straight hair', 'afro', 'buzz cut', 'french twist', 'dreadlocks', 'fishtail braid', 'half-up half-down', 'side part', 'middle part', 'updo', 'loose curls', 'locks', 'layered cut', 'shag cut', 'crew cut', 'mohawk', 'bangs', 'chignon', 'top knot']
    appearances_words = ['eyes', 'face', 'skin', 'body', 'fur', 'chin', 'cheeks', 'big wolf', 'white wolf', 'beautiful', 'ugly', 'handsome', 'cute', 'gorgeous', 'sharp', 'features', 'arms', 'bicep', 'legs', 'ass', 'breasts', 'waist', 'muscular', 'pale', 'features', 'thin', 'fangs', 'tattoos', 'teeth', 'mouth', 'young', 'old', 'blood', 'bleeding', 'gaze', 'smirk', 'smile', 'lips', 'nose', 'hands', 'jaw']
    weather_words = ['morning', 'afternoon', 'evening', 'night', 'sunrise', 'sunset', 'dawn', 'dusk', 'noon', 'midnight', 'cloudy', 'rain', 'storm', 'wind', 'sun', 'sunny', 'fog', 'foggy', 'snow', 'snowy', 'hail', 'thunder', 'lightning', 'breeze', 'chilly', 'hot', 'warm', 'cold', 'frost', 'blizzard', 'temperature', 'humid', 'dry', 'drizzle', 'pouring', 'downpour', 'mist', 'overcast']

    # Поиск в главах по всем категориям
    clothes_results = search_in_all_chapters(chapters, clothes_words, 'Clothes')
    hair_results = search_in_all_chapters(chapters, hair_words, 'Hair')
    appearances_results = search_in_all_chapters(chapters, appearances_words, 'Appearances')
    weather_results = search_in_all_chapters(chapters, weather_words, 'Weather')

    # Объединяем все результаты
    all_results = clothes_results + hair_results + appearances_results + weather_results

    # Создаем DataFrame для всех данных
    all_data_df = pd.DataFrame(all_results, columns=['Chapter Number', 'Chapter Title', 'Match', 'Category'])

    # Сортируем по номеру главы (числовая сортировка)
    all_data_df.sort_values(by=['Chapter Number'], inplace=True)

    # Проверяем, существует ли директория
    os.makedirs(f"{chapters_folder}", exist_ok=True)
    
    # Сохраняем данные в один Excel файл
    excel_file_path = f"{chapters_folder}/details.xlsx"
    try:
        all_data_df.to_excel(excel_file_path, sheet_name='Details', index=False)
        print(f"Файл Excel создан: {excel_file_path}")
    except Exception as e:
        print(f"Ошибка при сохранении Excel файла: {e}")

# Определение папки для сохранения
chapters_folder = "stuck_with_alpha_mate - Copy"

# Вызов основной функции
docx_file_path = f"{chapters_folder}/fulltext.docx"
main(docx_file_path, chapters_folder)


Извлекаю текст из stuck_with_alpha_mate - Copy/fulltext.docx...
Пытаюсь открыть файл: stuck_with_alpha_mate - Copy/fulltext.docx
Файл успешно открыт.
Текст успешно извлечён.
Разделяю текст на главы...
Текст успешно разделён на 94 глав.
Ищу заданные слова для категории Clothes...
Найдено совпадений в 164 главах для категории Clothes.
Ищу заданные слова для категории Hair...
Найдено совпадений в 54 главах для категории Hair.
Ищу заданные слова для категории Appearances...
Найдено совпадений в 1276 главах для категории Appearances.
Ищу заданные слова для категории Weather...
Найдено совпадений в 265 главах для категории Weather.
Файл Excel создан: stuck_with_alpha_mate - Copy/details.xlsx
